In [1]:
import pandas as pd
import re 
import sqlite3
import numpy as np
import ast
import sklearn 
import math 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

this notebook preprocesses notes for Ben's text models 

In [2]:
conn = sqlite3.connect('lowva\lowva.db')
conn.text_factory = str
cur = conn.cursor()

# Concatenates notes for Ben to run through his TextCNN with neural word embeddings

In [18]:
dfcohort=pd.read_sql_query('''select pat_deid, lowvadate, outcome from outcome''', conn)
dfcohort["lowvadate"]=pd.to_datetime(dfcohort["lowvadate"])
dfcohort.head()

,pat_deid,lowvadate,outcome
0,1174,2009-09-17,1
1,1790,2012-02-14,0
2,2262,2012-05-15,0
3,2610,2009-11-19,1
4,2736,2016-07-11,1


In [9]:
dfcohort[dfcohort["pat_deid"]==3840966]

,pat_deid,lowvadate
5610,3840966,2015-03-05


In [4]:
dfnotes=pd.read_sql_query('''select * from notes''', conn) 
dfnotes["enc_date"]=pd.to_datetime(dfnotes["enc_date"])
dfnotes.head()
len(dfnotes)

,pat_deid,note_deid,enc_date,note,note_desc
0,1174,28528746.0,2009-01-12,"January 12, 2009 Peter R Egbert, MD R...",Outpatient Letter
1,1174,28528751.0,2009-01-12,Department of Ophthalmology California Vitreo...,RTF Letter
2,1174,58842312.0,2009-09-17,Assessment and Plan: 1. Proliferative diabeti...,Progress Notes
3,1790,174426007.0,2012-02-14,Retina Service Outpatient Encounter Steven ...,Progress Notes
4,1790,174435847.0,2012-02-14,Retina Service Outpatient Encounter Steven ...,RTF Letter


67690

In [11]:
#sort notes in backwards order 
dfnotes.sort_values(by=["pat_deid", "enc_date", "note_deid"], ascending=False, inplace=True) 
dfnotes

,pat_deid,note_deid,enc_date,note,note_desc
67689,3843115,396423434.0,2012-10-25,">> ELIZABETH SNODGRASS Thu Oct 25, 2012 3...",Nursing Note
67688,3843115,228201151.0,2012-10-25,10/25/2012 Patient Name: Richard Dumond Da...,RTF Letter
67687,3843115,228200494.0,2012-10-25,Call me immediately for any of the following: ...,Patient Instructions
67686,3843115,228200434.0,2012-10-25,"Theodore Leng, MD, FACS Clinical Assistant Pr...",Progress Notes
67685,3843115,208900992.0,2012-07-20,7/20/2012 Patient Name: Richard Dumond Dat...,RTF Letter
67684,3843115,208900954.0,2012-07-20,"Patient Examined: Richard Dumond, Date of Birt...",Progress Notes
67683,3843115,208900132.0,2012-07-20,Call me immediately for any of the following: ...,Patient Instructions
67682,3843115,189200962.0,2012-04-20,4/20/2012 Patient Name: Richard Dumond Dat...,RTF Letter
67681,3843115,189200916.0,2012-04-20,Call me immediately for any of the following: ...,Patient Instructions
67680,3843115,189200909.0,2012-04-20,"Theodore Leng, MD Clinical Assistant Professo...",Progress Notes


In [15]:
#drop missing notes 
dfnotes=dfnotes[~dfnotes["note"].isnull()]

In [16]:
#concatenate notes 
dfnotesconcat=dfnotes.groupby(['pat_deid'])['note'].apply(lambda x: '\n'.join(x)).reset_index()

In [17]:
dfnotesconcat

,pat_deid,note
0,1174,Assessment and Plan: 1. Proliferative diabeti...
1,1790,">> Tracy Vasquez, MA Tue Feb 14, 2012 2:4..."
2,2262,">> HUYNH VAN Tue May 15, 2012 11:55 AM Sl..."
3,2610,Trying to reprinter RX. Previous attempts wer...
4,2736,"Addendum created 07/20/16 0853 by Sigal, Evge..."
5,2920,Stanford University Hospital and Clinics Opht...
6,3178,"November 11, 2016 Rex C Hsei Bay Eye Medic..."
7,4062,"Charles Lin, MD Clinical Assistant Professor ..."
8,4690,Spoke with patient and he said that he needs 3...
9,5361,A/P 1) s/p bike accident with no ocular seque...


In [23]:
cohortnotes=pd.merge(dfcohort[["pat_deid", "outcome"]], dfnotesconcat, on="pat_deid", how="inner")

In [25]:
cohortnotes.to_csv("lowva-notesconcat.csv",index=False)

In [ ]:
conn.close()